In [1]:
import sys
# sys.path.append("/home/matthias/Documents/EmbeddedAI/deep-microcompression/")
sys.path.append("../../")

from development import (
    Sequential,
    AvgPool2d,
    BatchNorm2d,
    Conv2d,
    Linear,
    ReLU,
    ReLU6,
    MaxPool2d,
    Flatten, 

    EarlyStopper,
    quantize_per_tensor_assy,
    quantize_per_tensor_sy,
    dequantize_per_tensor_assy,
    dequantize_per_tensor_sy,
    
    QuantizationGranularity,
    QuantizationScaleType,
    QuantizationScheme
)

/home/matthias/Documents/EmbeddedAI/deep-microcompression/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os
import copy
import random

from tqdm.auto import tqdm

import torch
from torch import nn, optim
from torch.utils import data
from torchvision import datasets, transforms

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
lenet5_file = f"lenet5_state_dict_{DEVICE}.pth"
log_compression_details_file = "lenet5_compression_log.csv"

LUCKY_NUMBER = 25
torch.manual_seed(LUCKY_NUMBER)
torch.random.manual_seed(LUCKY_NUMBER)
torch.cuda.manual_seed(LUCKY_NUMBER)

DEVICE


'cpu'

In [4]:
input_shape = (1, 28, 28)

data_transform = transforms.Compose([
    transforms.RandomCrop((24, 24)),
    transforms.Resize(input_shape[1:]),
    transforms.ToTensor(),
])

mnist_train_dataset = datasets.MNIST("../../../Datasets/", train=True, download=True, transform=data_transform)
mnist_test_dataset = datasets.MNIST("../../../Datasets/", train=False, download=True, transform=data_transform)

mnist_train_loader = data.DataLoader(mnist_train_dataset, batch_size=32, shuffle=True, num_workers=os.cpu_count())
mnist_test_loader = data.DataLoader(mnist_test_dataset, batch_size=32, shuffle=False, num_workers=os.cpu_count())


In [5]:
lenet5_model = Sequential(
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, pad=tuple([2]*4), bias=False),
    # BatchNorm2d(num_features=6),
    ReLU6(),

    MaxPool2d(kernel_size=2, stride=2, padding=0),

    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True),
    # BatchNorm2d(num_features=16),
    ReLU(),
    # ReLU6(),

    MaxPool2d(kernel_size=2, stride=2, padding=0),
    # AvgPool2d(kernel_size=2, stride=2, padding=0),

    Flatten(),
    
    Linear(in_features=16*5*5, out_features=84, bias=False),
    ReLU6(),
    Linear(in_features=84, out_features=10, bias=True)
).to(DEVICE)

accuracy_fun = lambda y_pred, y_true: (y_pred.argmax(dim=1) == y_true).sum().item()




In [6]:
# lenet5_model = Sequential(
#     Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=5, pad=[2]*4, bias=True),
#     # BatchNorm2d(num_features=3),
#     ReLU(),
#     Conv2d(in_channels=6, out_channels=3, kernel_size=1, stride=1, pad=[0]*4, bias=False),
#     MaxPool2d(kernel_size=2, stride=2, padding=0),
#     Flatten(),
#     Linear(in_features=3*3*3, out_features=84, bias=False),
#     ReLU(),
#     Linear(in_features=84, out_features=10, bias=True)
# ).to(DEVICE)

# accuracy_fun = lambda y_pred, y_true: (y_pred.argmax(dim=1) == y_true).sum().item()

In [7]:
# lenet5_model = Sequential(
#     Conv2d(in_channels=1, out_channels=3, kernel_size=3, stride=5, pad=[2]*4, bias=False),
#     # BatchNorm2d(num_features=3),
#     ReLU(),
#     Flatten(),
#     Linear(in_features=3*6*6, out_features=10, bias=True)
# ).to(DEVICE)

# accuracy_fun = lambda y_pred, y_true: (y_pred.argmax(dim=1) == y_true).sum().item()

In [8]:
try:
    # raise RuntimeError
    lenet5_model.load_state_dict(torch.load(lenet5_file, weights_only=True), strict=True)
    
except (RuntimeError, FileNotFoundError) as e:
    early_stopper = EarlyStopper(
        metric_name="train_loss",
        min_valid_diff=1e-7,
        mode="min",
        patience=2,
        restore_best_state_dict=True,
    )

    criterion_fun = nn.CrossEntropyLoss()
    optimizion_fun = optim.Adam(lenet5_model.parameters(), lr=1.e-3)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

    lenet5_model.fit(
        mnist_train_loader, 2, 
        criterion_fun, optimizion_fun, lr_scheduler,
        validation_dataloader=mnist_test_loader, 
        metrics={"acc" : accuracy_fun},
        callbacks=[early_stopper],
        device=DEVICE
    )
    torch.save(lenet5_model.state_dict(), lenet5_file)
    

## Original Model


In [9]:
lenet5_mcu_model = copy.deepcopy(lenet5_model)

original_acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)
original_size = lenet5_mcu_model.get_size_in_bits()//8
print(original_acc*100)
print(f"The original model accuracy is {original_acc*100:.2f}% with size {original_size} bytes.")

# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(input_shape=input_shape, var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")

print(lenet5_mcu_model.test(device=DEVICE))
original_acc, original_size

  0%|          | 0/313 [00:00<?, ?it/s]

100%|██████████| 313/313 [00:03<00:00, 82.60it/s] 


99.33
The original model accuracy is 99.33% with size 148064 bytes.
tensor([[-10.5499,  -3.5355,   9.8397,  -3.2319,  -9.1715, -12.8788,  -4.4893,
           0.5832,  -4.2970,  -7.2553]])


(0.9933, 148064)

In [10]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
sp = .1
s = QuantizationScheme.DYNAMIC
g = QuantizationGranularity.PER_TENSOR
b = 4

# for i in range(0, 11):
#     sp = i/10

# for i in [8, 4, 2]:

#     b = i

RANGE = 10
for i in range(1000):
    sp = random.choice([i/RANGE for i in range(0, RANGE+1, 1)])
    s = random.choice([QuantizationScheme.NONE, QuantizationScheme.DYNAMIC, QuantizationScheme.DYNAMIC, QuantizationScheme.STATIC, QuantizationScheme.STATIC])
    g = random.choice([None, QuantizationGranularity.PER_CHANNEL, QuantizationGranularity.PER_TENSOR, QuantizationGranularity.PER_TENSOR, QuantizationGranularity.PER_TENSOR])
    b = random.choice([None, 2, 4, 4, 8, 8])

    print(sp, s, g, b)
    compression_config = {
        
        "prune_channel" :{
            "sparsity" : sp,
            "metric" : "l2"
        },

        "quantize" : {
            "scheme" : s,
            "granularity": g,
            "bitwidth" : b
        }

    }


        # compressed_lenet5_mcu_model = lenet5_mcu_model.init_compress(compression_config, input_shape=input_shape, calibration_data=next(iter(mnist_test_loader))[0].to(DEVICE))
    # lenet5_mcu_model.cpu()
    try:
        compressed_lenet5_mcu_model = lenet5_mcu_model.init_compress(compression_config, input_shape=input_shape, calibration_data=next(iter(mnist_test_loader))[0].to(DEVICE))
    except ValueError:
        continue
    compressed_lenet5_mcu_model.to(DEVICE)

    before_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100
    size = compressed_lenet5_mcu_model.get_size_in_bits()//8
    compressed_lenet5_mcu_model.to(DEVICE)

    print(f"Before training, sparsity = {sp}, scheme = {s}, granularity = {g}, bitwidth = {b} acc = {before_acc:.4f} size = {size/original_size*100:9.4f} {100 - size/original_size*100:9.4f}")
    # # compressed_lenet5_mcu_model.convert_to_c(input_shape=input_shape, var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")
    compressed_lenet5_mcu_model.to(DEVICE)
    # print(compressed_lenet5_mcu_model.test(device=DEVICE))

    # print("#"*40, "Training", "#"*40)
    # early_stopper = EarlyStopper(
    #     metric_name="validation_acc",
    #     min_valid_diff=.001,
    #     mode="min",
    #     patience=3,
    #     restore_best_state_dict=True,
    # )

    # criterion_fun = nn.CrossEntropyLoss()
    # # optimizion_fun = optim.SGD(compressed_lenet5_mcu_model.parameters(), lr=1.e-3)
    # optimizion_fun = optim.Adam(compressed_lenet5_mcu_model.parameters(), lr=1.e-3)
    # lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

    # compressed_lenet5_mcu_model.fit(
    #     mnist_train_loader, 
    #     15, 
    #     criterion_fun, optimizion_fun, lr_scheduler,
    #     validation_dataloader=mnist_test_loader, 
    #     metrics={"acc": accuracy_fun},
    #     verbose = True,
    #     device=DEVICE,
    #     callbacks = [early_stopper]
    # )
    # after_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100
    # print(f"After training, sparsity = {sp}, scheme = {s}, granularity = {g}, bitwidth = {b} acc = {after_acc:.4f}")

0.0 QuantizationScheme.STATIC QuantizationGranularity.PER_CHANNEL 2


0.9 QuantizationScheme.DYNAMIC QuantizationGranularity.PER_TENSOR 8


100%|██████████| 313/313 [00:05<00:00, 60.65it/s]


Before training, sparsity = 0.9, scheme = QuantizationScheme.DYNAMIC, granularity = QuantizationGranularity.PER_TENSOR, bitwidth = 8 acc = 13.7500 size =    0.4235   99.5765
0.3 QuantizationScheme.STATIC QuantizationGranularity.PER_TENSOR None
0.5 QuantizationScheme.STATIC QuantizationGranularity.PER_TENSOR 8


100%|██████████| 313/313 [00:08<00:00, 34.95it/s]

...


100%|██████████| 313/313 [00:05<00:00, 56.25it/s]


Before training, sparsity = 0.6, scheme = QuantizationScheme.DYNAMIC, granularity = QuantizationGranularity.PER_TENSOR, bitwidth = 8 acc = 31.0600 size =    4.6649   95.3351
0.3 QuantizationScheme.NONE QuantizationGranularity.PER_TENSOR 8
1.0 QuantizationScheme.NONE None 8
0.8 QuantizationScheme.NONE QuantizationGranularity.PER_CHANNEL 8
0.6 QuantizationScheme.DYNAMIC QuantizationGranularity.PER_CHANNEL None
0.2 QuantizationScheme.NONE QuantizationGranularity.PER_TENSOR 4
0.7 QuantizationScheme.STATIC QuantizationGranularity.PER_CHANNEL 8
0.0 QuantizationScheme.NONE None 4
0.6 QuantizationScheme.STATIC QuantizationGranularity.PER_TENSOR 8


100%|██████████| 313/313 [00:09<00:00, 33.25it/s]

Before training, sparsity = 0.6, scheme = QuantizationScheme.STATIC, granularity = QuantizationGranularity.PER_TENSOR, bitwidth = 8 acc = 27.0300 size =    4.6784   95.3216
0.7 QuantizationScheme.STATIC QuantizationGranularity.PER_TENSOR 2



 20%|█▉        | 62/313 [00:02<00:06, 38.08it/s]

In [ ]:
(input_scale * weight_scale).view(1, -1, 1, 1)
input_scale * weight_scale

100%|██████████| 313/313 [00:03<00:00, 92.59it/s] 


97.49

In [ ]:
s = 0.5
q = 3
b = 8

compression_config = {
    "prune_channel" :{
        "sparsity" : s
    },
    # "quantization" : {
    #     "type" : q,
    #     "bitwidth" : b
    # }

}

lenet5_mcu_model.cpu()
compressed_lenet5_mcu_model = lenet5_mcu_model.compress(compression_config, input_shape=input_shape, input_batch_real=next(iter(mnist_test_loader))[0])
compressed_lenet5_mcu_model.to(DEVICE)

before_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100

compressed_lenet5_mcu_model.cpu()
size = compressed_lenet5_mcu_model.get_size_in_bits()//8
compressed_lenet5_mcu_model.to(DEVICE)

print(f"Before training, sparsity = {s}, q_type = {q}, bitwidth = {b} acc = {before_acc:.4f} size = {size/original_size*100:9.4f}")
compressed_lenet5_mcu_model.cpu()
compressed_lenet5_mcu_model.convert_to_c(input_shape=input_shape, var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")
compressed_lenet5_mcu_model.to(DEVICE)
# print(compressed_lenet5_mcu_model.test(device=DEVICE))

early_stopper = EarlyStopper(
    metric_name="train_loss",
    min_valid_diff=1e-5,
    mode="min",
    patience=4,
    restore_best_state_dict=True,
)

criterion_fun = nn.CrossEntropyLoss()
# optimizion_fun = optim.SGD(compressed_lenet5_mcu_model.parameters(), lr=1.e-3)
optimizion_fun = optim.Adam(compressed_lenet5_mcu_model.parameters(), lr=10.e-3)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

compressed_lenet5_mcu_model.fit(
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    verbose = True,
    device=DEVICE,
    compression_config=compression_config,
    input_shape=input_shape, input_batch_real=next(iter(mnist_test_loader))[0],
    callbacks = [early_stopper]
)
after_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100
print(f"After training, sparsity = {s}, q_type = {q}, bitwidth = {b} acc = {after_acc:.4f}")



s = 0.5
q = 3
b = 4

compression_config = {
    "prune_channel" :{
        "sparsity" : s
    },
    "quantization" : {
        "type" : q,
        "bitwidth" : b
    }
}

compressed_lenet5_mcu_model.cpu()
compressed_lenet5_mcu_model = compressed_lenet5_mcu_model.compress(compression_config, input_shape=input_shape, input_batch_real=next(iter(mnist_test_loader))[0])
compressed_lenet5_mcu_model.to(DEVICE)

before_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100

compressed_lenet5_mcu_model.cpu()
size = compressed_lenet5_mcu_model.get_size_in_bits()//8
compressed_lenet5_mcu_model.to(DEVICE)

print(f"Before training, sparsity = {s}, q_type = {q}, bitwidth = {b} acc = {before_acc:.4f} size = {size/original_size*100:9.4f}")
compressed_lenet5_mcu_model.cpu()
compressed_lenet5_mcu_model.convert_to_c(input_shape=input_shape, var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")
compressed_lenet5_mcu_model.to(DEVICE)
# print(compressed_lenet5_mcu_model.test(device=DEVICE))

early_stopper = EarlyStopper(
    metric_name="train_loss",
    min_valid_diff=1e-5,
    mode="min",
    patience=4,
    restore_best_state_dict=True,
)

criterion_fun = nn.CrossEntropyLoss()
# optimizion_fun = optim.SGD(compressed_lenet5_mcu_model.parameters(), lr=1.e-3)
optimizion_fun = optim.Adam(compressed_lenet5_mcu_model.parameters(), lr=10.e-3)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

compressed_lenet5_mcu_model.fit(
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    verbose = True,
    device=DEVICE,
    compression_config=compression_config,
    input_shape=input_shape, input_batch_real=next(iter(mnist_test_loader))[0],
    callbacks = [early_stopper]
)
after_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100
print(f"After training, sparsity = {s}, q_type = {q}, bitwidth = {b} acc = {after_acc:.4f}")

AttributeError: 'Sequential' object has no attribute 'compress'

In [ ]:
compressed_lenet5_mcu_model.test(device=DEVICE), \
quantize_per_tensor_assy(
    compressed_lenet5_mcu_model.test(device=DEVICE),
    compressed_lenet5_mcu_model[-1].__dict__["_dmc"]["quantization"]["output_scale"],
    compressed_lenet5_mcu_model[-1].__dict__["_dmc"]["quantization"]["output_zero_point"]
)

(tensor([[ -4.2995,  -2.2290,  -1.3500,   7.7069,  -1.1915,  -1.9620, -12.2104,
           -2.9040,  -1.8079,  -1.4910]], device='cuda:0'),
 tensor([[-1,  0,  0,  4,  0,  0, -5,  0,  0,  0]], device='cuda:0',
        dtype=torch.int8))

In [ ]:
compressed_lenet5_mcu_model

Sequential(
  (conv2d_0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (relu6_0): ReLU6()
  (maxpool2d_0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_1): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu_0): ReLU()
  (maxpool2d_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten_0): Flatten(start_dim=1, end_dim=-1)
  (linear_0): Linear(in_features=400, out_features=84, bias=False)
  (relu6_1): ReLU6()
  (linear_1): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
compressed_lenet5_mcu_model.cpu()
test_input = compressed_lenet5_mcu_model.test_input.clone()

test_input_quant = quantize_per_tensor_assy(
    test_input,
    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_scale"],
    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_zero_point"],
    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["bitwidth"],
)

test_input_real = dequantize_per_tensor_assy(
    test_input_quant, 
    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_scale"],
    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_zero_point"],
)

i = 0
# print("original real", test_input[0,0,i])
# print("quant real", test_input_real[0,0,i])
# print("quant", test_input_quant[0,0,i])
# line = torch.clamp(test_input[0,0,i]/compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_scale"] + \
#                    compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_zero_point"], -128, 127)
# print(line)
# print((line- compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_zero_point"]) * compressed_lenet5_mcu_model.__dict__["_dmc"]["quantization"]["input_scale"])


conv0 = compressed_lenet5_mcu_model[0]
test_input_real = conv0(test_input_real)

relu0 = compressed_lenet5_mcu_model[1]
test_input_real = relu0(test_input_real)
i = 5*2
print(test_input_real.size())
print(test_input_real[0,0,i:i+2])
print(
    quantize_per_tensor_assy(
        test_input_real,
        conv0.__dict__["_dmc"]["quantization"]["output_scale"],
        conv0.__dict__["_dmc"]["quantization"]["output_zero_point"]
    )[0,0,i:i+2]
)
avgpool_0 = compressed_lenet5_mcu_model[2]
test_input_real = avgpool_0(test_input_real)

# flatten0 = compressed_lenet5_mcu_model[2]
# test_input_real = flatten0(test_input_real)

# linear0 = compressed_lenet5_mcu_model[3]
# test_input_real = linear0(test_input_real)

next_layer = compressed_lenet5_mcu_model[0]

i = 5
print(test_input_real.size())
print(test_input_real[0,0,i])
print(
    quantize_per_tensor_assy(
        test_input_real,
        next_layer.__dict__["_dmc"]["quantization"]["output_scale"],
        next_layer.__dict__["_dmc"]["quantization"]["output_zero_point"]
    )[0,0,i]
)

# pad_input = nn.functional.pad(
#         test_input_quant.to(torch.int32) - conv0.__dict__["_dmc"]["quantization"]["input_zero_point"], 
#         conv0.pad, 
#         "constant", 
#         conv0.__dict__["_dmc"]["quantization"]["input_zero_point"] - conv0.__dict__["_dmc"]["quantization"]["input_zero_point"]
#     )
# pad_weight = quant_weight.to(torch.int32)

# quant_weight = quantize_per_tensor_sy(
#         conv0.weight, 
#         conv0.__dict__["_dmc"]["quantization"]["weight_scale"],
#         conv0.__dict__["_dmc"]["quantization"]["bitwidth"]
#     )
# test_input_quant = nn.functional.conv2d(
#     nn.functional.pad(
#         test_input_quant.to(torch.int32) - conv0.__dict__["_dmc"]["quantization"]["input_zero_point"], 
#         conv0.pad, 
#         "constant", 
#         conv0.__dict__["_dmc"]["quantization"]["input_zero_point"] - conv0.__dict__["_dmc"]["quantization"]["input_zero_point"]
#     ),
#     quant_weight.to(torch.int32),
#     stride=5,
# )

# print(test_input_quant)
# print(test_input_quant*next_layer.__dict__["_dmc"]["quantization"]["bias_scale"])

# print(
#     quantize_per_tensor_assy(
#         test_input_quant*next_layer.__dict__["_dmc"]["quantization"]["bias_scale"],
#         next_layer.__dict__["_dmc"]["quantization"]["output_scale"],
#         next_layer.__dict__["_dmc"]["quantization"]["output_zero_point"]
#     )
# )

# print(test_input_real)
# print(
#     quantize_per_tensor_assy(
#         test_input_real,
#         next_layer.__dict__["_dmc"]["quantization"]["output_scale"],
#         next_layer.__dict__["_dmc"]["quantization"]["output_zero_point"]
#     )
# )

# print(
#     nn.functional.conv2d(
#         test_input_real, 
#         dequantize_per_tensor_sy(
#             quant_weight,
#                 conv0.__dict__["_dmc"]["quantization"]["weight_scale"],
#             ),
#         stride=5
#         # quantize_per_tensor_sy(
#         #     conv0.weight, 
#         #     conv0.__dict__["_dmc"]["quantization"]["weight_scale"],
#         #     conv0.__dict__["_dmc"]["quantization"]["bitwidth"]
#         # ).to(torch.int32),
#         # stride=5,
#     )

# )
# next_layer.__dict__["_dmc"]["quantization"]["output_zero_point"]
conv0.__dict__["_dmc"]["quantization"]["output_zero_point"]

torch.Size([1, 6, 28, 28])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]], grad_fn=<SliceBackward0>)
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3]], dtype=torch.int8)
torch.Size([1, 6, 14, 14])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<SelectBackward0>)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=torch.int8)


tensor(3, dtype=torch.int8)

## Pruned Model 

### sparsity_per_layer = 0.1

In [ ]:
 sparsity_per_layer = 0.25
RANGE = 10




if not os.path.exists(log_compression_details_file):
    with open(log_compression_details_file, "w") as file:
        file.write(f"sparsity, quantization_type, bitwidth, size, size_ratio, before acc, after acc, before acc_drop, after acc_drop\n")
        # file.write(f"sparsity, quantizaion_type, bitwidth, before acc, after acc\n")

for i in tqdm(range(1)):

    # s = random.choice([i/RANGE for i in range(0, RANGE+1, 1)])
    # q = random.choice([QUANTIZATION_NONE, DYNAMIC_QUANTIZATION_PER_TENSOR, DYNAMIC_QUANTIZATION_PER_TENSOR, DYNAMIC_QUANTIZATION_PER_TENSOR])
    # b = random.choice([4, 8])
    # print(f"sample number {i} ->  sparsity = {s}, q_type = {q}, bitwidth = {b}")

    s = 0.
    q = 1
    b = 8

    compression_config = {
        "prune_channel" :{
            "sparsity" : s
        },
        "quantization" : {
            "type" : q,
            "bitwidth" : b
        }

    }

    lenet5_mcu_model.cpu()
    compressed_lenet5_mcu_model = lenet5_mcu_model.compress(compression_config, input_shape=input_shape)
    compressed_lenet5_mcu_model.to(DEVICE)

    before_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100

    compressed_lenet5_mcu_model.cpu()
    size = compressed_lenet5_mcu_model.get_size_in_bits()//8
    compressed_lenet5_mcu_model.to(DEVICE)

    print(f"Before training, sparsity = {s}, q_type = {q}, bitwidth = {b} acc = {before_acc:.4f}")

    early_stopper = EarlyStopper(
        metric_name="train_loss",
        min_valid_diff=1e-5,
        mode="min",
        patience=4,
        restore_best_state_dict=True,
    )

    criterion_fun = nn.CrossEntropyLoss()
    optimizion_fun = optim.Adam(compressed_lenet5_mcu_model.parameters(), lr=1.e-3)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

    compressed_lenet5_mcu_model.fit(
        mnist_train_loader, 
        15, 
        criterion_fun, optimizion_fun, lr_scheduler,
        validation_dataloader=mnist_test_loader, 
        metrics={"acc": accuracy_fun},
        verbose = False,
        device=DEVICE,
        compression_config=compression_config,
        callbacks = [early_stopper]
    )
    after_acc = compressed_lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun, device=DEVICE)*100

    print(f"After training, sparsity = {i/RANGE:.2f}, bitwidth = {b} acc = {after_acc:.4f}")

    with open(log_compression_details_file, "a") as file:
        file.write(f"{s}, {q}, {b}, {size}, {size/original_size*100:9.4f}, {before_acc:9.4f}, {after_acc:9.4f}, {original_acc-before_acc:9.4f}, {original_acc-after_acc:9.4f}\n")
    # break

IndentationError: unexpected indent (1965599166.py, line 1)

In [ ]:
 sparsity_per_layer = 0.1
lenet5_model.to("cpu")
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
# acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
# print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.to(DEVICE)
lenet5_mcu_model.fit(
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device=DEVICE,
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.2

In [ ]:
sparsity_per_layer = 0.2
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.3

In [ ]:
sparsity_per_layer = 0.3
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.4

In [ ]:
sparsity_per_layer = 0.4
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.5

In [ ]:
sparsity_per_layer = 0.5
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.6

In [ ]:
sparsity_per_layer = 0.6
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.7

In [ ]:
sparsity_per_layer = 0.7
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.8

In [ ]:
sparsity_per_layer = 0.8
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### sparsity_per_layer = 0.9

In [ ]:
sparsity_per_layer = 0.9
lenet5_mcu_model = lenet5_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
# size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")
# print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.Adam(lenet5_mcu_model.parameters(), lr=1.e-2)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

lenet5_mcu_model.fit(
    # mnist_test_loader,
    mnist_train_loader, 
    15, 
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=mnist_test_loader, 
    metrics={"acc": accuracy_fun},
    device="cpu",
    compression_aware=True
)
lenet5_mcu_model = lenet5_mcu_model.prune_channel(sparsity_per_layer)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
print(f"The pruned model with sparsity {sparsity_per_layer} accuracy is {acc*100:.2f}%.")



# print(lenet5_mcu_model.test())
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


## Dynamic Quantized Per Tensor

### 8 bit quantization

In [ ]:
bitwidth = 8
lenet5_mcu_model = lenet5_model.dynamic_quantize_per_tensor(bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The dynamic quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


### 4 bit quantization

In [ ]:
bitwidth = 4
lenet5_mcu_model = lenet5_model.dynamic_quantize_per_tensor(bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The dynamic quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


## Dynamic Quantized Per Channel

### 8 bit quantization

In [ ]:
bitwidth = 8
lenet5_mcu_model = lenet5_model.dynamic_quantize_per_channel(bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The dynamic quantized per channel model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")


## Static Quantized Per Tensor

### 8 bit quantization

In [ ]:
bitwidth = 8
lenet5_mcu_model = lenet5_model.static_quantize_per_tensor(next(iter(mnist_test_loader))[0], bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The static quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")



### 4 bit quantization

In [ ]:
bitwidth = 4
lenet5_mcu_model = lenet5_model.static_quantize_per_tensor(next(iter(mnist_test_loader))[0], bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The static quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")



## Static Quantized Per Channel

### 8 bit quantization

In [ ]:
bitwidth = 8
lenet5_mcu_model = lenet5_model.static_quantize_per_channel(next(iter(mnist_test_loader))[0], bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The static quantized per channel model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")



### 4 bit quantization

In [ ]:
bitwidth = 4
lenet5_mcu_model = lenet5_model.static_quantize_per_channel(next(iter(mnist_test_loader))[0], bitwidth)
acc = lenet5_mcu_model.evaluate(mnist_test_loader, accuracy_fun)
size = lenet5_mcu_model.get_size_in_bits()//8
print(f"The static quantized per channel model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")
print(lenet5_mcu_model.test())
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
lenet5_mcu_model.convert_to_c(var_name="lenet5_mcu_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")



In [ ]:
# lenet5_model.cpu()

# # PRUNED MODEL
# pruned_sparsity = [i/10 for i in range(10)]
# for sparsity in pruned_sparsity:
#     pruned_model = lenet5_model.prune_channel(sparsity)
#     acc = pruned_model.evaluate(mnist_test_loader, accuracy_fun)
#     size = pruned_model.get_size_in_bits()//8
#     print(f"The pruned model with sparsity {sparsity} accuracy is {acc*100:.2f}%.")
#     print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

# quantization_bitwidth = [i for i in range(8, 0, -1)]

# # DYNAMIC QUANTIZED MODEL PER TERSON
# for bitwidth in quantization_bitwidth:
#     dynamic_quantized_per_tensor_model = lenet5_model.dynamic_quantize_per_tensor(bitwidth)
#     acc = dynamic_quantized_per_tensor_model.evaluate(mnist_test_loader, accuracy_fun)
#     size = dynamic_quantized_per_tensor_model.get_size_in_bits()//8
#     print(f"The dynamic quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
#     print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")


# # DYNAMIC QUANTIZED MODEL PER TERSON
# for bitwidth in quantization_bitwidth:
#     dynamic_quantized_per_channel_model = lenet5_model.dynamic_quantize_per_channel(bitwidth)
#     acc = dynamic_quantized_per_channel_model.evaluate(mnist_test_loader, accuracy_fun)
#     size = dynamic_quantized_per_channel_model.get_size_in_bits()//8
#     print(f"The dynamic quantized per channel model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
#     print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")


# # STATIC QUANTIZED MODEL PER TERSON
# for bitwidth in quantization_bitwidth:
#     static_quantized_per_tensor_model = lenet5_model.static_quantize_per_tensor(next(iter(mnist_test_loader))[0], bitwidth)
#     acc = static_quantized_per_tensor_model.evaluate(mnist_test_loader, accuracy_fun)
#     size = static_quantized_per_tensor_model.get_size_in_bits()//8
#     print(f"The static quantized per tensor model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
#     print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")


# # STATIC QUANTIZED MODEL PER TERSON
# for bitwidth in quantization_bitwidth:
#     static_quantized_per_channel_model = lenet5_model.static_quantize_per_channel(next(iter(mnist_test_loader))[0], bitwidth)
#     acc = static_quantized_per_channel_model.evaluate(mnist_test_loader, accuracy_fun)
#     size = static_quantized_per_channel_model.get_size_in_bits()//8
#     print(f"The static quantized per channel model with bitwidth {bitwidth} accuracy is {acc*100:.2f}%.")
#     print(f"The accurancy drop is {(original_acc - acc)*100:.2f}% and size drop is {(original_size - size)/original_size*100:.2f}%.")

